# H.7 Generalizing PREPARE and SELECT


In the last node, we found a circuit for applying an arbitrary linear combination of unitaries. It split naturally into two subroutines, PREPARE which prepared the state and coefficients, and SELECT which applied the unitaries. These were somewhat messily defined using nested circuits, but it turns there is a clean generalization of PREPARE and SELECT.

### Codercise H.7.1.
 Write a function for creating the Householder transformation as a matrix, assuming the input is a normalized state. You will find np.outer helpful.

In [ ]:
def householder(state):
    """Create the matrix form of a Householder transformation.
    
    Args:
        state (array[complex]): A list of amplitudes representing a state.

    Returns: 
        array[complex]: The matrix representation of the Householder transformation.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    return np.eye(state.shape[0])- 2*np.outer(state,state)



### Codercise H.7.2. 
(a) Implement the PREPARE oracle using the Householder transformation; this is available from the previous exercise. For simplicity, we work with a sum of  unitaries. This means our auxiliary register will have  qubits, and the state  is the all-zero state (usually denoted ).

In [ ]:
k_bits = 2
n_bits = 2
all_bits = k_bits + n_bits
aux = range(k_bits)
main = range(k_bits, all_bits)
dev = qml.device("default.qubit", wires=all_bits)

def PREPARE(alpha_list):
    """Create the PREPARE oracle as a matrix.
    
    Args:
        alpha_list (array[float]): A list of coefficients.

    Returns: 
        array[complex]: The matrix representation of the PREPARE routine.
    """
    zero_vec = np.array([1] + [0]*(2**k_bits - 1))
    ##################
    # YOUR CODE HERE #
    ##################
    alpha_list = np.sqrt(alpha_list) / np.linalg.norm(alpha_list)
    return householder((zero_vec - alpha_list) / np.linalg.norm(zero_vec -alpha_list))


(b) The circuit below shows a PREPARE oracle (red), followed by a SELECT oracle (blue), followed by PREPARE (red):

![](https://codebook.xanadu.ai/pics/ps-oracles.svg)

In [ ]:
def SELECT(U_list):
    """Implement the SELECT oracle for 2^k unitaries."""
    for index in range(2**k_bits):
        ctrl_str = np.binary_repr(index, k_bits) # Create binary representation
        qml.ControlledQubitUnitary(U_list[index], control_wires=aux, 
                                   wires=main, control_values=ctrl_str)

def LCU(alpha_list, U_list):
    """Implement LCU using PREPARE and SELECT oracles for 2^k unitaries.
    
    Args:
        alpha_list (list[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices, stored as 
        complex arrays.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.QubitUnitary(PREPARE(alpha_list), wires=aux)
    SELECT(U_list)
    qml.adjoint(qml.QubitUnitary)(PREPARE(alpha_list), wires=aux)


(c) Finally, consider the combination of unitaries

In [ ]:
U_list = [np.kron(qml.PauliX.compute_matrix(), qml.PauliX.compute_matrix()),
          np.kron(qml.PauliZ.compute_matrix(), qml.PauliZ.compute_matrix()),
          np.kron(qml.PauliX.compute_matrix(), qml.PauliZ.compute_matrix()),
          np.kron(qml.PauliZ.compute_matrix(), qml.PauliX.compute_matrix())]
alpha_list = [1, 0.5, 0.5, 1]

@qml.qnode(dev)
def my_circuit():
    """Apply H(X + Z/2) to the state |11> using LCU."""
    ##################
    # YOUR CODE HERE #
    ##################
    for _ in main:
	    qml.PauliX(wires=_)
    LCU(alpha_list, U_list)
    return qml.state()

print("The amplitudes on the main register are proportional to", my_circuit()[:4], ".")


### Codercise H.7.3. 
Use the PREPARE and SELECT procedures you defined above to generate superpositions of two-qubit computational basis states,

In [ ]:
@qml.qnode(dev)
def quantum_memory(beta_list):
    """Produce a data state with positive coefficients beta_list.

    Args:
        beta_list (array[float]): The amplitudes for our superposition.

    Returns: 
        array[float]: The state on both address and data registers.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    bell = np.matmul(np.kron(qml.Hadamard.compute_matrix(), np.eye(2)),qml.CNOT.compute_matrix())
    
    U_list = [np.kron(qml.PauliZ.compute_matrix(), qml.PauliZ.compute_matrix()),
    np.kron(qml.PauliZ.compute_matrix(),qml.PauliX.compute_matrix()),
    np.kron(qml.PauliX.compute_matrix(),qml.PauliZ.compute_matrix()),
    np.kron(qml.PauliX.compute_matrix(),qml.PauliX.compute_matrix())]
    beta_sq = np.square(beta_list)

    qml.QubitUnitary(PREPARE(beta_sq), wires=aux)
    SELECT(U_list)



    return qml.state()

beta_list = np.array([1, 0, 0, 1])
normalized_coefficients = [quantum_memory(beta_list)[i].item() for i in range(0, 20, 5)]
print("The amplitudes on the main register are proportional to", normalized_coefficients, ".")
